# COVID-19 Worldwide report analysis

Utilizes the daily data reporting from Johns Hopkins University Center for Systems Science and Engineering (JHU CSSE):
https://systems.jhu.edu/. This is pulled from time series maintained at Github repo:
https://github.com/CSSEGISandData/COVID-19.  

Using the introductory Data Science Tables (for reference see http://data8.org/datascience/tables.html) of introductory courses.

On 3/23 the timeseries moved and US state level data is no longer present.

In [ ]:
# HIDDEN
# This useful nonsense should just go at the top of your notebook.
from datascience import *
%matplotlib inline
#%matplotlib notebook
import matplotlib.pyplot as plots
import numpy as np
import scipy
plots.style.use('fivethirtyeight')
plots.rc('lines', linewidth=2, color='r')
from ipywidgets import interact
import ipywidgets as widgets
# datascience version number of last run of this notebook
version.__version__

'0.15.0'

In [ ]:
import sys
sys.path.append(".")
from timetable import TimeTable

import locale
locale.setlocale( locale.LC_ALL, 'en_US.UTF-8' ) 

import os
import datetime

# Parsing and cleaning
def denan(v):
    return v if v != 'nan' else np.nan

def clean(tbl):
    for lbl in tbl.labels:
        tbl[lbl] = tbl.apply(denan, lbl)

def is_state(name):
    return not ',' in name

def is_county(name):
    return ',' in name

def getstate(name):
    county, state = name.split(', ')
    return state

def getcounty(name):
    county, state = name.split(', ')
    return county

# Tools for working with timestamps
def less_day(day1, day2):
    """Return day1 < day2"""
    return datetime.datetime.strptime(day1, "%m/%d/%y") < datetime.datetime.strptime(day2, "%m/%d/%y")

# Projecting growth rates

def ave_growth(trend, window=4):
    """Average recent growth rate of single trend"""
    vals = [x for x in trend.take[-window:]['rate'] if np.isfinite(x)]
    return scipy.stats.gmean(vals)

def inc_day(day, ndays=1):
    """Return day + ndays"""
    date =  datetime.datetime.strptime(day, "%m/%d/%y") + datetime.timedelta(days=ndays)
    return datetime.datetime.strftime(date, "%m/%d/%y")

def format_day(day):
    """Return day """
    date =  datetime.datetime.strptime(day, "%m/%d/%y")
    return datetime.datetime.strftime(date, "%m/%d/%y")

def project_trend(trend, num_days, rate=None):
    if rate :
        growth_rate = rate
    else :
        growth_rate = ave_growth(trend)
        
    day = trend.last('Day')
    val = trend.last(1)
    growth = trend.last('new')
    pnew = trend.last('% new')
    proj = trend.copy()
    for i in range(num_days):
        day = inc_day(day)
        growth = round(growth * growth_rate)
        val = val + growth
        pnew = growth/val
        proj.append((day, val, growth, pnew, growth_rate))
    return proj

In [ ]:
# Tools for working with content
def by_country(raw_world):
    """Aggregate country level data from the raw source"""
    res = raw_world.drop(['Province/State', 'Lat', 'Long']).group('Country/Region', sum)
    for lbl in res.labels[1:] :
        res.relabel(lbl, format_day(lbl[:-4]))
    return res

In [ ]:
# CSV files for data currated by JHU.
# These changed recently

#confirmedURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv"
confirmedURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv" 

#deathURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv"
#recoveredURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv"

In [ ]:
# Raw data of confirmed cases
raw_confirmed = Table.read_table(confirmedURL)
raw_confirmed

In [ ]:
# Aggregate data by country

raw_by_country = by_country(raw_confirmed)

### How many of the 195 recognized countries in the world have reported cases?

This does include some non-countries, like Princess Cruises

In [ ]:
raw_by_country.num_rows

In [ ]:
# Transpose country data to provide timeseries column per country

countries_by_day = TimeTable.transpose(raw_by_country, 'Country/Region', time_col='Day', time_less = less_day)
countries_by_day['Day'] = countries_by_day.apply(format_day, 'Day')
#countries_by_day.take[-7:]

In [ ]:
# Change this to truncate data analysys to an earlier date
last_day = countries_by_day.last('Day')
print("last day of data:", last_day)
#last_day = "3/22/20"
countries_by_day = countries_by_day.until(last_day)
countries_by_day.order_cols().take[-10:]

### Total confirmed cases worldwide

In [ ]:
total_confirmed = countries_by_day.select('Day')
total_confirmed['Worldwide'] = countries_by_day.sum_rows()
print('Total confirmed', total_confirmed.last('Worldwide'))
total_confirmed.obar(height=6, width=8)
_ = plots.xticks(rotation=45)

## Global confirmed cases by country - largest

In [ ]:
countries_by_day.stackbar(15, height=6, width=8)
_ = plots.xticks(rotation=45)

## Global picture sans China

The global growth rate above is somewhat optimistic, since the growth across the world is amortized over the substantial, but flat, cases in China, now mostly resolved.  Removing that we see a more accurate picture of the trends

In [ ]:
# Recent worldwide growth trend - past week
total_confirmed.trend().take[-7:]

In [ ]:
# Taking China out of the picture
sans_china_by_day = countries_by_day.drop('China')
sans_china_by_day.stackbar(15, height=6, width=7)
_ = plots.xticks(rotation=45)

In [ ]:
sans_china_confirmed = sans_china_by_day.select('Day')
sans_china_confirmed['Worldwide sans China'] = sans_china_by_day.sum_rows()
sans_china_confirmed.trend().take[-15:].show()

## Projecting global trends two weeks out

The following indicates confirmed cases for the two weeks ahead.

In [ ]:
project_trend(total_confirmed.trend().take[-7:], 14).show()

In [ ]:
proj = project_trend(total_confirmed.trend().take[-10:], 14).select(['Day', 'Worldwide', 'new'])
proj.bar('Day')

Assuming China stays flat and using the growth rate of the rest of the world, we get a more concerning picture.

In [ ]:
project_trend(sans_china_confirmed.trend().take[-10:], 14).show()

In [ ]:
proj = project_trend(sans_china_confirmed.trend().take[-10:], 14).select(range(3))
proj.bar('Day')

# Country level trends

Recent confirmed cases on a country by country basis.

In [ ]:
countries_by_day.top(15).oplot(height=6, width=9)
xs = countries_by_day['Day']
_ = plots.xticks(xs[range(0, len(xs), 5)], rotation=45)

In [ ]:
def cases_since(cases, threshold=100, width=6, height=6):
    _, axis = plots.subplots(figsize=(width, height))
    for region in cases.categories :
        ctbl = cases.extract(region)
        since = ctbl.where(ctbl[region] >= threshold)
        ndays = since.num_rows
        vals = since[region]
        axis.plot(vals)
        axis.text(ndays-1, vals[-1], region)

In [ ]:
cases_since(countries_by_day.top(10), width=8)

In [ ]:
countries_by_day.top(15).take[-10:]

In [ ]:
countries_by_day.top(15).trend().take[-10:]

In [ ]:
def project_one(country, back, forward):
    return project_trend(countries_by_day.extract(country).take[-back:].trend(), forward).select('Day', country)

def project_all(back, forward):
    projs = project_one(countries_by_day.categories[0], back, forward)
    for country in countries_by_day.categories[1:] :
        try :
            proj = project_one(country, back, forward)
            if not np.isnan(proj.last(country)) :
                projs[country] = proj[country]
        except :
            print('skip', country)
    return projs

In [ ]:
ww_projection = project_all(7, 14).order_cols()
ww_projection['Day'] = ww_projection.apply(format_day, 'Day')
ww_projection.show()

In [ ]:
ww_projection.stackbar(20, height=8, width=8)

In [ ]:
ww_projection.top(15).oplot(height=8, width=8)
_ = plots.xticks(rotation=45)

In [ ]:
countries_by_day.top(10).take[-7:]

In [ ]:
final_trend = countries_by_day.trend().take[-1:]
final_trend

In [ ]:
def countries(raw_world):
    """Country level metadata from the raw source"""
    res = raw_world.select(['Country/Region', 'Lat', 'Long']).group('Country/Region', np.mean)
    return res

def get_new(trend, country):
    return trend['new ' + country][-1]

def get_rate(trend, country):
    return trend['rate ' + country][-1]

In [ ]:
days = countries_by_day.num_rows
country_summary = countries(raw_confirmed).join('Country/Region', raw_by_country.select(['Country/Region', last_day]))
country_summary['new'] = country_summary.apply(lambda c: get_new(final_trend, c), 'Country/Region')
country_summary['growth'] = country_summary['new'] / country_summary[last_day]
country_summary['rate'] = country_summary.apply(lambda c: get_rate(final_trend, c), 'Country/Region')
country_summary['days'] = country_summary.apply(lambda c: days - np.count_nonzero(countries_by_day[c] < 5), 'Country/Region')

In [ ]:
country_summary.sort('rate', descending=True).show()

In [ ]:
def label_point(country, x, y):
    t = country_summary.where('Country/Region', country)
    plots.text(t[x][0], t[y], country)

## Growth rate versus number of confirmed cases

A greast deal of attention is focused on the countries with the largest number of confirmed cases.  But that mostly refects the the time since community transmission started.  We should be paying more attention to growth rates.  That paints a very different picture.  The large infected population is increasing around 10% per day.  But many of the countries that are earlier in the process are growing incredibly quickly.

In [ ]:
largest_cases = country_summary.sort(last_day, descending=True).take[:8]
largest_cases

In [ ]:
largest_growth = country_summary.sort('growth', descending=True).take[:10]
largest_growth

In [ ]:
country_summary.where(country_summary['growth']>=0).select([last_day, 'growth']).scatter('growth', width=8)
for c in largest_cases['Country/Region']:
    label_point(c, 'growth', last_day)
label_point('Turkey', 'growth', last_day)

In [ ]:
country_summary.select([last_day, 'days']).scatter('days', width=8)
for c in largest_cases['Country/Region']:
    label_point(c, 'days', last_day)

In [ ]:
country_summary.select(['growth', 'days']).scatter('days', width=8)
for c in largest_growth['Country/Region']:
    label_point(c, 'days', 'growth')

In [ ]:
country_summary.sort('days', descending=True).take[:15].show()

## Will warmer seasons help?  What about lower lattitudes?

Much has been said about warmer weather reducing the spread.  Some have suggested that southern hemisphere or lower latitudes are harbingers of how that might develop.  We can look at confirmed cases and growth by latitude.


In [ ]:
country_summary.where(country_summary['growth']>=0).select([last_day, 'Lat mean']).scatter(last_day, width=8)
for c in largest_cases['Country/Region']:
    label_point(c, last_day, 'Lat mean')
max_cases = max(country_summary[last_day])
plots.plot([0,max_cases], [23.5, 23.5])
plots.plot([0,max_cases], [-23.5, -23.5])

In [ ]:
country_summary.where(country_summary['growth'] > 0.2)

In [ ]:
country_summary.where(country_summary['growth']>=0).select(['growth', 'Lat mean']).scatter('growth', width=8, height=9)
for c in country_summary.where(country_summary['growth'] > 0.125)['Country/Region']:
    label_point(c, 'growth', 'Lat mean')
max_growth = max(country_summary['growth'])
_ = plots.plot([0,max_growth], [23.5, 23.5])
_ = plots.plot([0,max_growth], [-23.5, -23.5])

### Confirmed cases in one country

In [ ]:
w = widgets.Dropdown(
    options=countries_by_day.categories,
    value='US',
    # rows=10,
    description='Country:',
    disabled=False
)
w

In [ ]:
country = w.value

In [ ]:
country_trend = countries_by_day.extract(country).trend()
country_trend.following('3/11/20').show()

In [ ]:
recent = country_trend.following('3/11/20')
recent.extract([country, 'new']).bar('Day', height=5)
_ = plots.xticks(rotation=45)

In [ ]:
projection = project_trend(recent, 14)
projection.show()

In [ ]:
projection.extract([country, 'new']).bar('Day')
_ = plots.xticks(rotation=45)